## Gold

## Load

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

## 01_trips_time_per_membertype

In [ ]:
# Databricks notebook source
from gold import *


GOLD_TABLE = "divvy.gold_trips_time_per_membertype"

spark.sql(f"DROP TABLE IF EXISTS {GOLD_TABLE};")

df = spark.table("divvy.fact_trips")
rider_df = spark.table("divvy.dim_riders")

# COMMAND ----------

df = df.join(rider_df, df.rider_id == rider_df.rider_id, "inner")
df = (
    df.groupby(df.is_member)
    .agg(
        F.avg(df.time_spent).alias("avg_time_spent"),
        F.sum(df.time_spent).alias("sum_time_spent"),
    )
    .select(
        F.col("is_member"),
        F.col("avg_time_spent"),
        F.col("sum_time_spent"),
    )
)

df.write.format("delta").mode("overwrite").saveAsTable(GOLD_TABLE)


## 02_trips_time_per_membertype

In [ ]:
from gold import *


GOLD_TABLE = "divvy.gold_trips_time_per_stations"

spark.sql(f"DROP TABLE IF EXISTS {GOLD_TABLE};")

df = spark.table("divvy.fact_trips")


# COMMAND ----------

df = (
    df.groupby(df.start_station_id, df.end_station_id)
    .agg(
        F.avg(df.time_spent).alias("avg_time_spent"),
        F.sum(df.time_spent).alias("sum_time_spent"),
    )
    .select(
        F.col("start_station_id"),
        F.col("end_station_id"),
        F.col("avg_time_spent"),
        F.col("sum_time_spent"),
    )
)


## 03_payments_per_date

In [ ]:
from gold import *


GOLD_TABLE = "divvy.gold_payments_per_date"

spark.sql(f"DROP TABLE IF EXISTS {GOLD_TABLE};")

df = spark.table("divvy.fact_payments")
date_df = spark.table("divvy.dim_date")

df = df.join(date_df, df.date == date_df.date, "inner")


# COMMAND ----------

df = (
    df.groupby(df.year, df.quarter_of_year, df.month)
    .agg(
        F.avg(df.amount).alias("avg_amount"),
        F.sum(df.amount).alias("sum_amount"),
    )
    .orderBy("year", "quarter_of_year", "month")
    .select(
        F.col("year"),
        F.col("quarter_of_year"),
        F.col("month"),
        F.col("avg_amount"),
        F.col("sum_amount"),
    )
)

df.write.format("delta").mode("overwrite").saveAsTable(GOLD_TABLE)

## 04_payments_per_age

In [ ]:
from gold import *

GOLD_TABLE = "divvy.gold_payments_per_age"

spark.sql(f"DROP TABLE IF EXISTS {GOLD_TABLE};")

df = spark.table("divvy.fact_payments")

# COMMAND ----------

df = (
    df.groupby(df.rider_age_account_start)
    .agg(
        F.avg(df.amount).alias("avg_amount"),
        F.sum(df.amount).alias("sum_amount"),
    )
    .orderBy(df.rider_age_account_start)
    .select(
        F.col("rider_age_account_start"),
        F.col("avg_amount"),
        F.col("sum_amount"),
    )
)

df.write.format("delta").mode("overwrite").saveAsTable(GOLD_TABLE)

## 05_trips_per_date

In [ ]:
from gold import *

GOLD_TABLE = "divvy.gold_trips_per_date"

spark.sql(f"DROP TABLE IF EXISTS {GOLD_TABLE};")

df = spark.table("divvy.fact_trips")

# COMMAND ----------

date_df = spark.table("divvy.dim_date")

df = df.join(date_df, df.start_at_date == date_df.date, "inner")
df = df.withColumn("time_15_min", F.floor(F.minute(df.start_at) / 15) + 1)

df = (
    df.groupby(df.day_of_week, df.time_15_min)
    .agg(
        F.avg(df.time_spent).alias("avg_time_spent"),
        F.sum(df.time_spent).alias("sum_time_spent"),
    )
    .orderBy(df.day_of_week, df.time_15_min)
    .select(
        F.col("day_of_week"),
        F.col("time_15_min"),
        F.col("avg_time_spent"),
        F.col("sum_time_spent"),
    )
)

df.write.format("delta").mode("overwrite").saveAsTable(GOLD_TABLE)


## 06_payments_per_rides

In [ ]:
from gold import *

GOLD_TABLE = "divvy.gold_payments_per_rides"

spark.sql(f"DROP TABLE IF EXISTS {GOLD_TABLE};")

df = spark.table("divvy.fact_payments")
date_df = spark.table("divvy.dim_date").select(
    F.col("date"),
    F.col("month"),
)
df = df.join(date_df, df.date == date_df.date, "inner")


df = (
    df.groupby(df.rider_id, df.month)
    .agg(
        F.count(df.rider_id).alias("count_rides"),
        F.avg(df.amount).alias("avg_amount"),
        F.sum(df.amount).alias("sum_amount"),
    )
    .orderBy("rider_id", "month")
    .select(
        F.col("rider_id"),
        F.col("month"),
        F.col("avg_amount"),
        F.col("sum_amount"),
        F.col("count_rides"),
    )
)

df.write.format("delta").mode("overwrite").saveAsTable(GOLD_TABLE)


## 07_payments_per_rides_minutes

In [ ]:
from gold import *

GOLD_TABLE = "divvy.gold_payments_per_rides_minutes"

spark.sql(f"DROP TABLE IF EXISTS {GOLD_TABLE};")

df = spark.table("divvy.fact_payments")
date_df = spark.table("divvy.dim_date").select(
    F.col("date"),
    F.col("month"),
    F.col("year"),
)
df = df.join(date_df, df.date == date_df.date, "inner")

# COMMAND ----------

# Refactored code
trips_df = spark.table("divvy.fact_trips")
trips_df = trips_df.join(date_df, trips_df.start_at_date == date_df.date, "inner")
trips_df = (
    trips_df.groupby(trips_df.rider_id, trips_df.month, trips_df.year)
    .agg(
        F.sum(trips_df.time_spent).alias("sum_time_spent"),
    )
    .selectExpr("rider_id AS trips_rider_id", "month AS trips_month", "year AS trips_year", "sum_time_spent")
    .orderBy("rider_id", "year", "month")
)

# COMMAND ----------

df = (
    df.groupby(df.rider_id, df.month, df.year)
    .agg(
        F.sum(df.amount).alias("sum_amount"),
    )
    .orderBy("rider_id", "year", "month")
    .select(
        F.col("rider_id"),
        F.col("month"),
        F.col("year"),
        F.col("sum_amount"),
    )
)
conds = [
    df.rider_id == trips_df.trips_rider_id,
    df.month == trips_df.trips_month,
    df.year == trips_df.trips_year,
]
df = df.join(trips_df, conds, "inner").select(
    F.col("rider_id"),
    F.col("month"),
    F.col("year"),
    F.col("sum_amount"),
    F.col("sum_time_spent"),
)

df.write.format("delta").mode("overwrite").saveAsTable(GOLD_TABLE)
